In [ ]:
import numpy as np
import zipfile

# 1. Unzip your submission
ZIP_NAME = 'weights_before_dates.zip'
EXTRACT_DIR = 'predictions/mlp'

with zipfile.ZipFile(ZIP_NAME, 'r') as z:
    z.extractall(EXTRACT_DIR)

# 2. Load predictions
preds_1 = np.load(f'{EXTRACT_DIR}/preds_1.npy')
preds_2 = np.load(f'{EXTRACT_DIR}/preds_2.npy')

# 3. Sanity‐check shapes
if preds_1.shape != (1000, 28):
    raise ValueError(f'preds_1 has size {preds_1.shape}, but expected (1000, 28)')
if preds_2.shape != (1818, 28):
    raise ValueError(f'preds_2 has size {preds_2.shape}, but expected (1818, 28)')

# 4. Define weighted log-loss
def weighted_log_loss(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """
    Compute weighted cross-entropy loss:
      L = - mean_i [ w_{y_i} * log(p_{i, y_i}) ]
    where w_c = 1 / f_c, then normalized to sum to 1 over classes.
    """
    # class frequencies and weights
    class_counts = y_true.sum(axis=0)                # shape (28,)
    class_weights = 1.0 / class_counts               # inverse freq
    class_weights /= class_weights.sum()             # normalize

    # per-sample weight = weight of the true class
    sample_weights = (y_true * class_weights).sum(axis=1)  # shape (N,)

    # log-prob of the true class for each sample
    log_ps = (y_true * np.log(y_pred)).sum(axis=1)        # shape (N,)

    # weighted average negative log‐likelihood
    return - np.mean(sample_weights * log_ps)

# 5. (Optional) Dummy one-hot labels so code runs end-to-end
y_test_1_oh = (np.arange(28) == np.random.choice(28, size=1000)[:, None]).astype(int)
y_test_2_oh = (np.arange(28) == np.random.choice(28, size=1818)[:, None]).astype(int)

# 6. Compute and print losses
loss_1 = weighted_log_loss(y_test_1_oh, preds_1)
loss_2 = weighted_log_loss(y_test_2_oh, preds_2)

print(f'Weighted log-loss for test set 1: {loss_1:.6f}')
print(f'Weighted log-loss for test set 2: {loss_2:.6f}')


Weighted log-loss for test set 1: 0.222536
Weighted log-loss for test set 2: 0.213751
